In [1]:
from slimtp.pipelines import AMPreprocessing
from omegaconf import OmegaConf
from hydra import compose, initialize
from slimtp.modules import AMDataset
from dataset import dino_amdataset

import pandas as pd
import numpy as np

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
with initialize(version_base=None, config_path="../../slimtp_configs/living_area/configs/", job_name="test_app"):
    cfg = compose(config_name="config")

In [4]:
ampreproc = AMPreprocessing(cfg)
ampreproc.run()

======== Stage AMPreprocessing ========
100%|██████████| 10/10 [00:00<00:00, 65.88it/s]
[2023-12-21 03:12:15][slimtp.pipelines.am_preprocessing][WARNING] - Drop 1 uncacheable samples.
/root/cape/capelibs/mtp/slimtp/lib/data.py:206: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['geometry', 'imagery_source', 'split', 'use', 'cache_key'], dtype='object')]

  df.to_hdf(path, key="data", mode="w")


In [5]:
df = pd.read_hdf(cfg.data_index_path)
df["cache_key"] = df["cache_key"].apply(eval)

In [6]:
class DinoInferenceDataset(AMDataset):
    def __getitem__(self, index: int):
        item = self.get_df_data(index)
        sample_name = item["identifier"]
        x_clean = self.get_aug_item(item, self.t_clean)
        return x_clean, sample_name


In [7]:
amdataset_1 = DinoInferenceDataset(cfg, df)

In [8]:
amdataset_2 = dino_amdataset.DinoInferenceDataset(cfg, df)

In [9]:
image_1, ident_1 = amdataset_1[0]

In [10]:
image_2, ident_2 = amdataset_2[0]

In [12]:
image_1

tensor([[[ 0.0176, -0.0255, -0.0647,  ..., -0.0725, -0.0725, -0.0882],
         [-0.0333, -0.0412, -0.0529,  ..., -0.0882, -0.0922, -0.1039],
         [-0.0255, -0.0294, -0.0294,  ..., -0.1039, -0.1000, -0.1275],
         ...,
         [-0.1745, -0.2216, -0.2725,  ..., -0.1784, -0.1392, -0.2333],
         [-0.2216, -0.2137, -0.2529,  ..., -0.1471, -0.1314, -0.2804],
         [-0.3549, -0.3353, -0.3039,  ..., -0.0686, -0.1902, -0.3863]],

        [[-0.0333, -0.0725, -0.1078,  ..., -0.1275, -0.1275, -0.1471],
         [-0.0843, -0.0843, -0.0961,  ..., -0.1471, -0.1510, -0.1627],
         [-0.0686, -0.0765, -0.0765,  ..., -0.1588, -0.1549, -0.1824],
         ...,
         [-0.2333, -0.2765, -0.3235,  ..., -0.1353, -0.1000, -0.2020],
         [-0.2686, -0.2608, -0.3000,  ..., -0.1078, -0.0922, -0.2490],
         [-0.3980, -0.3824, -0.3471,  ..., -0.0294, -0.1549, -0.3549]],

        [[-0.0020, -0.0412, -0.0804,  ..., -0.1745, -0.1745, -0.1824],
         [-0.0529, -0.0529, -0.0647,  ..., -0

In [14]:
image_2

tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.8044,  ..., -1